
@Author:Vijay Kumar M N<br>
@Date: 2024-10-24<br>
@Last Modified by:Vijay Kumar M N<br>
@Last Modified: 2024-10-24<br>
@Title :Python Program to interact with rds and import and export<br>

In [1]:
import pymssql
import csv
import boto3
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Fetch credentials from the environment
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.getenv('AWS_DEFAULT_REGION')

# Function to connect to MSSQL RDS
def connect_to_database(endpoint, port, username, password, database):
    conn = pymssql.connect(server=f"{endpoint}:{port}", user=username, password=password, database=database)
    return conn

# Function to export data from MSSQL to a CSV file
def export_data_to_csv(conn, table_name, csv_file):
    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    
    # Write data to CSV
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i[0] for i in cursor.description])  # Write headers
        writer.writerows(cursor.fetchall())  # Write data
    
    print(f"Data exported to {csv_file}")

# Function to upload the CSV file to S3
def upload_file_to_s3(file_name, bucket_name, s3_file_name):
    # Initialize the boto3 client using environment credentials
    s3 = boto3.client('s3',
                      aws_access_key_id=AWS_ACCESS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                      region_name=AWS_REGION)

    try:
        s3.upload_file(file_name, bucket_name, s3_file_name)
        print(f"File uploaded to S3: {bucket_name}/{s3_file_name}")
    except FileNotFoundError:
        print("The file was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

if __name__ == "__main__":
    # MSSQL RDS connection details
    endpoint = 'database-1.c3e2sig42eps.ap-south-1.rds.amazonaws.com'
    port = 1433
    username = 'admin'
    password = 'vijaykumar123'
    database = 'vijayvk'

    # CSV and S3 details
    table_name = 'Employees'
    csv_file = 'employees_export.csv'
    bucket_name = 'backupsql1'
    s3_file_name = 'employees_export.csv'

    # Connect to MSSQL and export data to CSV
    conn = connect_to_database(endpoint, port, username, password, database)
    try:
        export_data_to_csv(conn, table_name, csv_file)
        
        # Upload the CSV to S3
        upload_file_to_s3(csv_file, bucket_name, s3_file_name)
    finally:
        conn.close()


Data exported to employees_export.csv
File uploaded to S3: backupsql1/employees_export.csv


In [18]:
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.getenv('AWS_DEFAULT_REGION')

# Function to connect to MSSQL RDS
def connect_to_database(endpoint, port, username, password, database):
    conn = pymssql.connect(server=f"{endpoint}:{port}", user=username, password=password, database=database)
    return conn

# Function to download CSV from S3
def download_file_from_s3(bucket_name, s3_file_name, local_file_name):
    s3 = boto3.client('s3',
                      aws_access_key_id=AWS_ACCESS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                      region_name=AWS_REGION)
    try:
        s3.download_file(bucket_name, s3_file_name, local_file_name)
        print(f"File downloaded from S3: {s3_file_name}")
    except FileNotFoundError:
        print("The file was not found.")
    except NoCredentialsError:
        print("Credentials not available.")
    except Exception as e:
        print(f"Error downloading file: {e}")

# Function to import data from CSV into MSSQL
def import_data_from_csv(conn, table_name, csv_file):
    cursor = conn.cursor()

    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Get header row
        num_columns = len(headers)  # Count columns

        for row in reader:
            if len(row) != num_columns:  # Ensure row matches header length
                print(f"Skipping row due to mismatched column count: {row}")
                continue

            try:
                # Assuming the first column is the primary key, adjust column names if necessary
                cursor.execute(f"""
                    IF NOT EXISTS (SELECT * FROM {table_name} WHERE id = %s)  -- Replace 'id' with your primary key column name
                    BEGIN
                        INSERT INTO {table_name} VALUES ({', '.join(['%s'] * num_columns)})  -- Use the correct number of placeholders
                    END
                """, (row[0],) + tuple(row))  # Adjust this based on your primary key and number of columns
            except pymssql.IntegrityError as e:
                print(f"Duplicate entry for primary key {row[0]}: {e}")
            except Exception as e:
                print(f"Error inserting row {row}: {e}")

    conn.commit()
    print(f"Data imported from {csv_file}")

# Function to list objects in the S3 bucket for troubleshooting
def list_objects_in_s3(bucket_name):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:
        print(f"Objects in {bucket_name}:")
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("Bucket is empty or does not exist.")

if __name__ == "__main__":
    # MSSQL RDS connection details
    endpoint = 'database-1.c3e2sig42eps.ap-south-1.rds.amazonaws.com'
    port = 1433
    username = 'admin'
    password = 'vijaykumar123'
    database = 'vijayvk'

    # S3 and CSV details
    bucket_name = 'backupsql1'
    s3_file_name = 'employees_export.csv'
    local_file_name = 'C:/Users/vijay/OneDrive/Desktop/sql/employees_import.csv'  # Adjust path if needed
    table_name = 'Employees1'

    # List objects in S3 for troubleshooting
    list_objects_in_s3(bucket_name)

    # Download CSV from S3
    download_file_from_s3(bucket_name, s3_file_name, local_file_name)

    # Check if the file exists before importing
    if os.path.exists(local_file_name):
        # Connect to MSSQL and import data from CSV
        conn = connect_to_database(endpoint, port, username, password, database)
        try:
            import_data_from_csv(conn, table_name, local_file_name)
        finally:
            conn.close()
    else:
        print(f"File {local_file_name} not found after download.")


Objects in backupsql1:
employees_export.csv
File downloaded from S3: employees_export.csv
Data imported from C:/Users/vijay/OneDrive/Desktop/sql/employees_import.csv
